### Esercitazione su Word Sense Disambiguation: 

#### INPUT : 
l’input per questa esercitazione è costituito da coppie di termini contenute nel file WordSim353 (disponibile nei
formati .tsv e .csv)
- Il file contiene 353 coppie di termini utilizzati come testset in varie competizioni internazionali
- A ciascuna coppia è attribuito un valore numerico [0,10], che rappresenta la similarità fra gli elementi della coppia.

#### Prima parte

Implementare tre misure di similarità basate su WordNet. Le misure sono: Wu & Palmer, Shortest Path e Leakcock & Chodorow. 
- per ciascuna di tali misure di similarità, calcolare
    - gli indici di correlazione di Spearman e
    - gli indici di correlazione di Pearson fra i risultati ottenuti e quelli ‘target’ presenti nel file annotato.

### Seconda parte

Implementare l’algoritmo di Lesk (!= usare implementazione esistente, e.g., in nltk…).
1. Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di
WN) e disambiguare (almeno) un sostantivo per frase. 
Calcolare l’accuratezza del sistema implementato sulla base dei sensi annotati in
SemCor.
    - SemCor è disponibile all’URL http://web.eecs.umich.edu/~mihalcea/downloads.html
2. Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, 
e restituire l’accuratezza media su (per esempio) 10 esecuzioni del programma.